In [727]:
import pandas_datareader as web
import matplotlib.pyplot as plt
import pandas as pd
from dask.distributed import Client
from dask.distributed import as_completed
import pandas_market_calendars as mcal
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RepeatedKFold, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm

from dask.distributed import Client
from dask.distributed import as_completed
from IPython.display import clear_output
import time 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from sklearn.model_selection import TimeSeriesSplit

from statsmodels.tsa.statespace.sarimax import SARIMAX


from dask.distributed import Client
from dask.distributed import as_completed

pd.set_option("max_rows", 10)

start = "1970-01-01"

In [728]:
#sp500_ancient = web.DataReader("^GSPC", "yahoo",start=start)
sp500 = web.DataReader("^GSPC", "yahoo",start=start)
vix = web.DataReader("VIXCLS","fred",start=start)
T10Y3M = web.DataReader("T10Y3M","fred",start=start)
FEDFUNDS = web.DataReader("FEDFUNDS","fred",start=start)


In [729]:
def ts_cv_kfolds(dataindex,k,sized=False,tratio=2):
    divisor = len(dataindex)/(k+1)
    train_ = []
    test_ = []
    
    for x in range(1,k+1,1):
        if(sized):
            train_size = int(np.round(divisor*x+divisor/tratio,0))
            test_size = int(np.round(divisor*x/x/tratio,0))
            #total_size = train_size+test_size
            #train_size = int(np.round(total_size*tproportion))
            #test_size = total_size-train_size
            train = dataindex[int(np.round(divisor*(x-1),0)):train_size]
            test = dataindex[train_size:train_size+test_size]
            train_.append(train)
            test_.append(test)
        #growing
        else:   
            train_size = int(np.round(divisor*x+divisor/tratio,0))
            test_size = int(np.round(divisor*x/x/tratio,0))
            train = dataindex[0:train_size]
            test = dataindex[train_size:train_size+test_size]
            train_.append(train)
            test_.append(test)
    return [train_, test_]

In [730]:
data_ = sp500[['Adj Close']].asfreq('Q',method='Pad')

k_=5

#cv_outer = KFold(n_splits=k_, shuffle=False)
#cv_outer = train_test_split(data_, test_size=0.3, random_state=0,shuffle=False,)

#tscv_outer = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(data_)/k_),test_size=int(len(data_)/k_))

outer_results = []
outer_models = []

outer_sets = []
#for trainv_ix, test_ix in tscv_outer.split(data_):
#for trainv_ix, test_ix in cv_outer.split(data_):
#for trainv_ix, test_ix in cv_outer.split(data_):

data_index = ts_cv_kfolds(data_.index,k_,True)

for di in range(0,len(data_index[0]),1):
    trainv_ix = data_index[0][di]
    test_ix = data_index[1][di]
    
    # split data
    X_trainv, X_test = data_.loc[trainv_ix], data_.loc[test_ix]
    outer_sets.append([X_trainv, X_test])
    #y_trainv, y_test = y.iloc[trainv_ix], y.iloc[test_ix]


In [ ]:
[o[0] for o in outer_sets]

In [772]:
def inner_arima(test_sets,model_=False):
    
    k_=3
    
    #cv_inner = KFold(n_splits=k_, shuffle=False)
    #tscv_inner = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(test_sets[0])/k_),test_size=int(len(test_sets[0])/k_))    
    
    X_trainv = test_sets[0]
    X_test = test_sets[1]
    #print(X_test.index)
    
    inner_results = list()
    inner_models = list()

    #for trainv_ix, test_ix in ts_cv_kfolds(data_.index,k_):
    
    # split data
    
    #X_trainv, X_test = data_.loc[trainv_ix], data_.loc[test_ix]
    
    #good juncture to fork locally
    #for train_ix, valid_ix in tscv_inner.split(X_trainv):
    #for train_ix, valid_ix in cv_inner.split(X_trainv):
    data_index = ts_cv_kfolds(X_trainv.index,k_,True)
    
    for di in range(0,len(data_index[0]),1):
        train_ix = data_index[0][di]
        valid_ix = data_index[1][di]
        
        # split data
        #print(train_ix)
        X_train, X_valid = X_trainv.loc[train_ix], X_trainv.loc[valid_ix]
        #print(X_train)
        #print(X_valid)
        #y_train, y_valid = y_trainv.iloc[train_ix], y_trainv.iloc[valid_ix]
        
        if (model_==False):
            #print(np.shape(X_train))
            #model = pm.auto_arima(X_train, seasonal=True, m=4,stepwise=True)
            model = pm.auto_arima(X_train, seasonal=True, m=4)
            forecasts = model.predict(len(X_valid))  # predict N steps into the future
        else:
            #else:
            model = SARIMAX(X_train,order=model_.order,seasonal_order=model_.seasonal_order, intercept=model_.with_intercept)
            #model.fit(disp=0)
            #forecasts = model.predict(start=len(X_train)+1,end=len(X_train)+len(X_valid))
            
            #model_fit = model.fit(start=0, end=len(X_train),exog=X_train,disp=False)
            model_fit = model.fit(disp=False)
            
            forecasts = model_fit.predict(start=len(model_fit.fittedvalues)+1, end=len(model_fit.fittedvalues)+len(X_valid),dynamic=True)
            #print(forecasts)
            #print(X_valid)
            #sm.tsa.statespace.SARIMAX(X_train, seasonal=True, m=4, order=my_order, seasonal_order=my_seasonal_order, ...)
            #pm.auto_arima(X_train, seasonal=True, m=4,)
            
        #plt.figure(figsize  = (8,8))
        #plt.title('Forecast result and Test data')
        # Visualize the forecasts (blue=train, green=forecasts)
        #x = np.arange(y.shape[0])
        #plt.plot(x[:150], train, c='blue')
        #plt.plot(x[150:], forecasts, c='green', label = 'ARIMA Forecast')
        #plt.plot(x[150:], test, c='red', label = 'Test Data')
        #plt.legend()
        #plt.show()
        
        inner_models.append(model)

        #yhat = model.predict(X_valid[t])
        yhat = forecasts
        # evaluate the model
        acc = np.mean(abs(X_valid.values.ravel()-forecasts))
        # store the result
        inner_results.append(acc)
        # report progress
        #print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
        # summarize the estimated performance of the model

    #good juncture to fork locally
    model_scores = list()

    #test inner model as ansemble
    for m_ in range(0,len(inner_models),1):
        m = inner_models[m_]
        X_valid = data_index[1][m_]
        
        if(str(type(m))=="<class 'statsmodels.tsa.statespace.sarimax.SARIMAX'>"):
            
            #model_fit = m.fit(disp=0)
            model_fit = m.fit(disp=False)
            forecasts = model_fit.predict(start=len(model_fit.fittedvalues)+1,end=len(X_test)+len(X_valid))

            predict = model_fit.predict(start=(len(model_fit.fittedvalues)+len(X_valid)+1),end=(len(model_fit.fittedvalues)+len(X_valid)+len(X_test)))
            #print(predict)
            model_scores.append(np.mean(abs(predict)-X_test.values.ravel()))
        else:
            
            model_scores.append(np.mean(abs(m.predict(len(X_test))-X_test.values.ravel())))

    #average internal model results and report as outer_result which*
    return([np.mean(np.abs(model_scores)),inner_models])
    #outer_results.append(np.mean(np.abs(model_scores)))
    #outer_models.append(inner_models)

    #print('I Accuracy: %.3f (%.3f)' % (mean(inner_results), std(inner_results)))


In [ ]:
outer_sets

In [758]:
inner_arima(outer_sets[0])

[79.3215060773795,
 [ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
        suppress_warnings=True),
  ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 4),
        suppress_warnings=True, with_intercept=False),
  ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(1, 1, 0, 4),
        suppress_warnings=True, with_intercept=False)]]

In [759]:
client = Client('192.168.3.100:8786')
#client.restart()

future = client.map(inner_arima, outer_sets)

results = client.gather(future)

In [760]:
results

[[79.3215060773795,
  [ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
         suppress_warnings=True),
   ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 4),
         suppress_warnings=True, with_intercept=False),
   ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(1, 1, 0, 4),
         suppress_warnings=True, with_intercept=False)]],
 [188.7966012535078,
  [ARIMA(order=(0, 0, 2), scoring_args={}, seasonal_order=(2, 1, 0, 4),
         suppress_warnings=True),
   ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 4),
         suppress_warnings=True),
   ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
         suppress_warnings=True, with_intercept=False)]],
 [541.8351189883024,
  [ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
         suppress_warnings=True, with_intercept=False),
   ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(1, 0, 0, 4),
         suppress_warnings=True)

In [761]:
np.mean([r[0] for r in results])

646.850736288278

In [762]:
results[0][1][0].seasonal_order

(0, 0, 0, 4)

In [763]:
results[0][1][0].order

(1, 0, 0)

In [ ]:
#dir(results[0][1][0])

In [ ]:
#outer_sets[0],results[0][1][0]

#model_ = pm.auto_arima(data_, seasonal=True, m=4)



In [ ]:
#forecasts = model_.predict(5)  # predict N steps into the future

In [736]:
model = SARIMAX(data_,order=model_.order,seasonal_order=model_.seasonal_order, intercept=model_.with_intercept)
#model.fit(disp=0)
#forecasts = model.predict(start=len(X_train)+1,end=len(X_train)+len(X_valid))

model_fit = model.fit(start=0, end=len(data_),exog=data_,disp=False)
forecasts = model_fit.predict(start=len(data_)+1, end=len(data_)+10,dynamic=True)

/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: end, exog, start. After release 0.14, this will raise.
  warnings.warn(
/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/tsa/statespace/kalman_filter.py:2290: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'


In [749]:
model.fit(disp=False)

/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [755]:
len(data_)

209

In [756]:
len(model_fit.fittedvalues)

209

In [773]:
inner_arima(outer_sets[0],results[0][1][0])

/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/tsa/statespace/kalman_filter.py:2290: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'
/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/tsa/statespace/kalman_filter.py:2290: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'
/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/mnt/distvol/py39_jupyterlab/lib/python3.9/site-packages/statsmodels/tsa/statespace/kalman_filter.py:2290: ValueWarning: Dynamic prediction specified to begin during out-of-sample for

[102.07585859728299,

In [774]:
strings = []
for i in list(matplotlib.cbook.flatten([r[1] for r in results])):
    strings.append(str(i))

In [775]:
strings.sort()

[' ARIMA(0,0,2)(2,1,0)[4] intercept',
 ' ARIMA(0,1,0)(0,0,0)[4]          ',
 ' ARIMA(0,1,0)(0,0,0)[4]          ',
 ' ARIMA(0,1,0)(0,0,0)[4] intercept',
 ' ARIMA(0,1,0)(1,0,0)[4] intercept',
 ' ARIMA(0,1,0)(1,1,0)[4]          ',
 ' ARIMA(1,0,0)(0,0,0)[4] intercept',
 ' ARIMA(1,0,0)(0,1,0)[4]          ',
 ' ARIMA(1,0,0)(0,1,0)[4] intercept',
 ' ARIMA(1,0,1)(0,0,0)[4] intercept',
 ' ARIMA(1,1,0)(0,0,0)[4] intercept',
 ' ARIMA(1,1,0)(1,0,0)[4] intercept',
 ' ARIMA(2,0,0)(0,1,0)[4]          ',
 ' ARIMA(2,0,0)(0,1,0)[4]          ',
 ' ARIMA(2,0,0)(2,1,0)[4] intercept']

In [776]:
[l.order for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]
[l.seasonal_order for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]
[l for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]

[ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
       suppress_warnings=True),
 ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 4),
       suppress_warnings=True, with_intercept=False),
 ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(1, 1, 0, 4),
       suppress_warnings=True, with_intercept=False),
 ARIMA(order=(0, 0, 2), scoring_args={}, seasonal_order=(2, 1, 0, 4),
       suppress_warnings=True),
 ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 4),
       suppress_warnings=True),
 ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
       suppress_warnings=True, with_intercept=False),
 ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 4),
       suppress_warnings=True, with_intercept=False),
 ARIMA(order=(0, 1, 0), scoring_args={}, seasonal_order=(1, 0, 0, 4),
       suppress_warnings=True),
 ARIMA(order=(1, 1, 0), scoring_args={}, seasonal_order=(1, 0, 0, 4),
       suppress_warnings=T

In [ ]:
strings

In [778]:
string_counts = []

for u in np.unique(strings):
    #print(u)
    count = np.count_nonzero(np.array(strings)==u)
    string_counts.append([u,count])
    #print(np.array(strings)==u)

In [779]:
[s[0] for s in string_counts]

[' ARIMA(0,0,2)(2,1,0)[4] intercept',
 ' ARIMA(0,1,0)(0,0,0)[4]          ',
 ' ARIMA(0,1,0)(0,0,0)[4] intercept',
 ' ARIMA(0,1,0)(1,0,0)[4] intercept',
 ' ARIMA(0,1,0)(1,1,0)[4]          ',
 ' ARIMA(1,0,0)(0,0,0)[4] intercept',
 ' ARIMA(1,0,0)(0,1,0)[4]          ',
 ' ARIMA(1,0,0)(0,1,0)[4] intercept',
 ' ARIMA(1,0,1)(0,0,0)[4] intercept',
 ' ARIMA(1,1,0)(0,0,0)[4] intercept',
 ' ARIMA(1,1,0)(1,0,0)[4] intercept',
 ' ARIMA(2,0,0)(0,1,0)[4]          ',
 ' ARIMA(2,0,0)(2,1,0)[4] intercept']

In [780]:
sorted_string_counts_df = pd.DataFrame(string_counts,columns=['arima model','times occurred in nested cv']).sort_values(by='times occurred in nested cv',ascending=False)

In [782]:
len(sorted_string_counts_df)

13

In [781]:
sorted_string_counts_df.head(10)

,arima model,times occurred in nested cv
1,"ARIMA(0,1,0)(0,0,0)[4]",2
11,"ARIMA(2,0,0)(0,1,0)[4]",2
0,"ARIMA(0,0,2)(2,1,0)[4] intercept",1
2,"ARIMA(0,1,0)(0,0,0)[4] intercept",1
3,"ARIMA(0,1,0)(1,0,0)[4] intercept",1
4,"ARIMA(0,1,0)(1,1,0)[4]",1
5,"ARIMA(1,0,0)(0,0,0)[4] intercept",1
6,"ARIMA(1,0,0)(0,1,0)[4]",1
7,"ARIMA(1,0,0)(0,1,0)[4] intercept",1
8,"ARIMA(1,0,1)(0,0,0)[4] intercept",1


In [ ]:
inner_arima(outer_sets[0],

In [ ]:
strings==np.unique(strings)